In [1]:
import pybamm
from src import DiffusionOnly, Simulation

In [5]:
model = DiffusionOnly()

parameters = pybamm.ParameterValues({
    "Electrode diffusivity [m2.s-1]": 1,
    "Electrolyte diffusivity [m2.s-1]": 1,
    "Reaction rate constant [A.m-2]": 1,
    "Maximum concentration [mol.m-3]": 1,
    "Initial electrode concentration [mol.m-3]": 0.5,
    "Initial electrolyte concentration [mol.m-3]": 0.5,
    "Transfer coefficient": 0.5,
    "Electrode thickness [m]": 1,
    "Electrolyte thickness [m]": 1,
})

sim = Simulation(model, parameter_values=parameters)
sol = sim.solve([0, 100])

At t = 3.53264 repeated recoverable right-hand side function errors.


SolverError: Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [CvodesInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/cvodes_interface.cpp:392: CVode returned "CV_REPTD_RHSFUNC_ERR". Consult CVODES documentation.

In [3]:
sol.plot(["Electrode concentration [mol.m-3]", "Electrolyte concentration [mol.m-3]"])

interactive(children=(FloatSlider(value=0.0, description='t', max=0.001, step=1e-05), Output()), _dom_classes=…